In [4]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from pyoculus.problems.toybox import psi_maxwellboltzmann, psitob
from pyoculus.problems.toybox import psitob, psi_circularcurrentloop
from pyoculus.problems.toybox import A_r_squared, rot, A_squared

from jax import jit, jacfwd
import jax.numpy as jnp


In [5]:
def equ_squared(rr, R, Z, sf, shear):
    """
    Returns the B field derived from the Psi and F flux functions derived with the fluxes:
    $$
        \psi = (z-Z)^{2} + \left(R - r\right)^{2}
        F = \left(2 sf + 2 shear \left(z^{2} + \left(R - r\right)^{2}\right)\right) \sqrt{R^{2} - z^{2} - \left(R - r\right)^{2}}
    $$
    using the relation B = grad x A, with A_\phi = \psi / r and B_\phi = F / r for an axisymmetric field in cylindrical coordinates.
    """
    temp = jnp.maximum(R**2 - (R - rr[0]) ** 2 - (Z - rr[2]) ** 2, 0.0)
    sgn = (1 + jnp.sign(R**2 - (R - rr[0]) ** 2 - (Z - rr[2]) ** 2)) / 2
    return sgn * jnp.array(
        [
            -(-2 * Z + 2 * rr[2]) / rr[0],
            (2 * sf + 2 * shear * ((R - rr[0]) ** 2 + (Z - rr[2]) ** 2))
            * jnp.sqrt(temp)
            / rr[0] ** 2,
            (-2 * R + 2 * rr[0]) / rr[0],
        ]
    )

In [6]:
equ_squared([8.,0.,0.], 6., 0., 0.91, 0.7)

Array([-0.        ,  0.65584154,  0.5       ], dtype=float64)

In [7]:
da = jacfwd(A_r_squared)(jnp.array([11.999,0.,0.]), 6., 0., 0.91, 0.7)
da

Array([0.        , 0.        , 0.47656771], dtype=float64, weak_type=True)

In [8]:
b = jit(rot(A_squared))

b(jnp.array([8.,0.,0.]), 6., 0., 0.91, 0.7)

Array([0.        , 0.65584154, 0.5       ], dtype=float64)

In [9]:
%timeit b(jnp.array([8.,0.,0.]), 6., 0., 0.91, 0.7)

85.4 µs ± 3.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
b2 = jit(equ_squared)

b2([8.,0.,0.], 6., 0., 0.91, 0.7)

Array([-0.        ,  0.65584154,  0.5       ], dtype=float64)

In [11]:
%timeit b2(jnp.array([6.,0.,0.]), 6., 0., 0.91, 0.7)

110 µs ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
A_z_ellipse(jnp.array([6.,0.,1.]), 6., 0., 0.91, 0.7, 1, 1)

In [ ]:
A_z_squared(jnp.array([6.,0.,1.]), 6., 0., 0.91, 0.7)

In [ ]:
da/8

In [ ]:
b = jit(psitob(psi_circularcurrentloop))

In [ ]:
b([8.,0.,0.], 6., 0)

In [ ]:
field_circularcurrentloop([8.,0.,0.], 6., 0)

In [ ]:
-0.13783259/-0.04387348

In [ ]:
b = jit(psitob(psi_maxwellboltzmann))
dbdx = jit(jacfwd(psitob(psi_maxwellboltzmann)))

In [ ]:
b([6. , 0., 0.], 6, 0., 1.41, 3, 1)

In [ ]:
dbdx(jnp.array([6. , 0., 0.]), 6, 0., 1.41, 3, 1)

In [ ]:
np.isnan(np.array([b([6.41409781+R , 0.02956604772482006, -0.69367863], 6.41409781, -0.69367863, 1.41, 3, 1) for R in np.logspace(-400, 2, 200)])).sum(axis=1)

In [ ]:
np.isnan(np.array([dbdx([6.41409781+R , 0.02956604772482006, -0.69367863], 6.41409781, -0.69367863, 1.41, 3, 1) for R in np.logspace(-400, 2, 200)])).sum(axis=2).sum(axis=1)

In [ ]:
from pyoculus.problems import AnalyticCylindricalBfield

separatrix = {"type": "circular-current-loop", "amplitude": -10, "R": 6, "Z": -5.5}
maxwellboltzmann = {"m": 6, "n": -1, "d": np.sqrt(2), "type": "maxwell-boltzmann", "amplitude": 1e-3}

# Creating the pyoculus problem object, adding the perturbation here use the R, Z provided as center point
pyoproblem = AnalyticCylindricalBfield.without_axis(
    6,
    0,
    0.91,
    0.6,
    perturbations_args=[separatrix],
    Rbegin=1,
    Rend=8,
    niter=800,
    guess=[6.41, -0.7],
    tol=1e-9,
)
# # Adding perturbation after the object is created uses the found axis as center point
pyoproblem.add_perturbation(maxwellboltzmann, find_axis=False)

In [ ]:
from pyoculus.integrators import RKIntegrator

iparams = dict()
iparams['rtol'] = 1e-10
iparams['ode'] = pyoproblem.f_RZ_tangent
iparams['verbosity'] = 2

integ = RKIntegrator(iparams)

In [ ]:
integ.set_initial_value(0, [pyoproblem._R0, pyoproblem._Z0, 1., 0., 0., 1.])

In [ ]:
integ.integrate(1)